# 기본 설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 952.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.4 MB/s eta 0:00:00
  Preparing metadata (se

In [3]:
#전처리 패키지
import pandas as pd
import numpy as np
from datetime import datetime
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
import spacy
import string

#토픽 모델링 패키지
from bertopic import BERTopic
import tensorflow_hub
from umap import UMAP
import hdbscan
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 데이터 전처리

In [9]:
df = pd.read_pickle('/content/drive/MyDrive/NH투자증권 공모전/data/2100번째_체크포인트.pkl')
df

,title,date,category,key_points,text,url
0,American teens aren't excited about virtual re...,2023-04-04,Tech,Virtual reality hasn't caught on with American...,A gallery assistant wearing an Oculus Quest 2 ...,https://www.cnbc.com/2023/04/04/american-teens...
1,Samsung launches new Galaxy S23 smartphone lin...,2023-02-01,Tech,Samsung launched its new Galaxy S23 smartphone...,Samsung on Wednesday launched its latest lineu...,https://www.cnbc.com/2023/02/01/samsung-galaxy...
2,"Chinese EV maker Xpeng expects cost cuts, Volk...",2023-08-21,Tech,Chinese EV maker Xpeng expects cost cuts and i...,Xpeng expects cost cuts and its Volkswagen par...,https://www.cnbc.com/2023/08/21/chinas-xpeng-e...
3,India is heading for 'very high growth' but ch...,2023-06-28,Asia Economy,India is on a high growth trajectory as more b...,"David Solomon, Chairman and CEO, Goldman Sachs...",https://www.cnbc.com/2023/06/28/india-offers-o...
4,Bed Bath & Beyond jumps 68% to lead last gasp ...,2023-01-11,Markets,,In this article BBBY\n\nAMC\n\nGME Follow your...,https://www.cnbc.com/2023/01/11/bed-bath-beyon...
...,...,...,...,...,...,...
2095,Europe stocks close higher as traders monitor ...,2023-05-19,,,The pan-European Stoxx 600 index closed up 0.7...,https://www.cnbc.com/2023/05/19/european-marke...
2096,Where wealthy investors are putting their cash...,2023-03-16,Markets,,Wealthy investors and family offices are movin...,https://www.cnbc.com/2023/03/16/where-wealthy-...
2097,Nasdaq drops more than 2% in worst day since F...,2023-08-01,,,The last time the U.S. got a downgrade from a ...,https://www.cnbc.com/2023/08/01/stock-market-t...
2098,‘Not just a fad’: Firm launches fund designed ...,2023-06-03,ETF Edge,,A major ETF firm provider is betting the artif...,https://www.cnbc.com/2023/06/03/artificial-int...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       2100 non-null   object        
 1   date        2100 non-null   datetime64[ns]
 2   category    2100 non-null   object        
 3   key_points  2100 non-null   object        
 4   text        2100 non-null   object        
 5   url         2100 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 98.6+ KB


In [11]:
#tech 관련 기사 추출
tech_df = df[df['category'].str.contains("Tech")]
tech_df

,title,date,category,key_points,text,url
0,American teens aren't excited about virtual re...,2023-04-04,Tech,Virtual reality hasn't caught on with American...,A gallery assistant wearing an Oculus Quest 2 ...,https://www.cnbc.com/2023/04/04/american-teens...
1,Samsung launches new Galaxy S23 smartphone lin...,2023-02-01,Tech,Samsung launched its new Galaxy S23 smartphone...,Samsung on Wednesday launched its latest lineu...,https://www.cnbc.com/2023/02/01/samsung-galaxy...
2,"Chinese EV maker Xpeng expects cost cuts, Volk...",2023-08-21,Tech,Chinese EV maker Xpeng expects cost cuts and i...,Xpeng expects cost cuts and its Volkswagen par...,https://www.cnbc.com/2023/08/21/chinas-xpeng-e...
6,Shares of Roblox fall 12% after company releas...,2023-04-17,Tech,Roblox's stock closed down 12% on Monday after...,"A child looks back at a banner for Roblox, dis...",https://www.cnbc.com/2023/04/17/roblox-stock-f...
7,Bluetooth tracker Tile is taking a new approac...,2023-02-16,Tech Drivers,"Tile, Life360's Bluetooth tagging device, is r...","""Theft is the primary reason people buy these ...",https://www.cnbc.com/2023/02/16/bluetooth-trac...
...,...,...,...,...,...,...
2062,Republican FTC Commissioner Wilson announces r...,2023-02-14,Tech,"Christine Wilson, the sole remaining Republica...","Christine Wilson, nominee to serve on the Fede...",https://www.cnbc.com/2023/02/14/republican-ftc...
2078,Amazon is focusing on using A.I. to get stuff ...,2023-05-15,Tech,Amazon is focusing on using artificial intelli...,Amazon is increasingly using robotics in its f...,https://www.cnbc.com/2023/05/15/amazon-is-focu...
2086,Tesla suppliers' shares jump as electric autom...,2023-01-06,Tech,Tesla previously cut prices in China in late O...,"Hong Kong, China, 13 Sept 2022, A red Tesla ca...",https://www.cnbc.com/2023/01/06/tesla-supplier...
2087,Nintendo Switch sales plunge 22% and the gamin...,2023-05-09,Tech,Nintendo reported a fall in profit and revenue...,Nintendo is hoping key games such as Mario and...,https://www.cnbc.com/2023/05/09/nintendo-full-...


In [14]:
tech_df['category'].unique()

array(['Tech', 'Tech Drivers', 'Clean Tech',
       'Technology Executive Council', 'Opinion - Tech'], dtype=object)

In [23]:
# 7월과 8월 데이터 필터링
july_august_data = tech_df[(tech_df['date'].dt.month == 7) | (tech_df['date'].dt.month == 8)]

# 결과 출력
july_august_data

,title,date,category,key_points,text,url
2,"Chinese EV maker Xpeng expects cost cuts, Volk...",2023-08-21,Tech,Chinese EV maker Xpeng expects cost cuts and i...,Xpeng expects cost cuts and its Volkswagen par...,https://www.cnbc.com/2023/08/21/chinas-xpeng-e...
10,Threads has lured some creators who never love...,2023-07-21,Tech Drivers,Creators and influencers are exploring whether...,In this article META Follow your favorite stoc...,https://www.cnbc.com/2023/07/21/instagram-thre...
16,HCA Healthcare patient data stolen and for sal...,2023-07-10,Tech,HCA Healthcare patient data has been hacked an...,"Sam Hazen, CEO of HCA Healthcare, speaks about...",https://www.cnbc.com/2023/07/10/hca-healthcare...
45,This Harvard dropout and her brother launched ...,2023-07-22,Clean Tech,"Annie, 22, and David Lu, 25, launched and are ...","David and Annie Lu, siblings and co-founders o...",https://www.cnbc.com/2023/07/22/-harvard-dropo...
52,"TikTok Shop strikes 'buy now, pay later' partn...",2023-07-21,Tech,"TikTok Shop, the e-commerce marketplace of the...","TikTok struck a partnership with ""buy now, pay...",https://www.cnbc.com/2023/07/21/tiktok-shop-in...
...,...,...,...,...,...,...
1986,China's chip metal export curbs are 'a wake up...,2023-07-13,Tech,China's metal export curbs on gallium and germ...,"Luisa Moreno, President of mining company Defe...",https://www.cnbc.com/2023/07/13/chinas-metal-c...
2028,Amazon to launch pay-by-palm technology at all...,2023-07-20,Tech,Amazon is bringing its pay-by-palm system to a...,Amazon is bringing its palm-scanning payment s...,https://www.cnbc.com/2023/07/20/amazon-to-add-...
2041,Apple supplier Foxconn's failed India chip ven...,2023-07-24,Tech,Foxconn is best known as the main assembler of...,In this article 2317-TW Follow your favorite s...,https://www.cnbc.com/2023/07/24/foxconn-failed...
2045,"Waymo's robotaxis coming to Austin, Texas",2023-08-03,Tech Drivers,"Waymo, the Alphabet-owned autonomous vehicle c...","Waymo, the Alphabet -owned autonomous vehicle ...",https://www.cnbc.com/2023/08/03/waymo-robotaxi...


In [ ]:
#text 추출
article_78 = july_august_data['text'].values
article_78

# 불용어 제거

In [24]:
#형태소 분석 모델 다운로드
!python -m spacy download en_core_web_md

2023-09-29 08:51:15.335493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [26]:
#형태소 분석 모델 정의 (spacy)
nlp = spacy.load("en_core_web_md")#trf 모델로 변경가능
spacy_stopwords_list = list(nlp.Defaults.stop_words)
nlp.max_length = 40000000

In [27]:
#불용어 리스트
print(spacy_stopwords_list)

['which', 'except', 'really', 'above', 'my', 'anyone', 'these', 'say', 'does', '’m', 'own', 'another', 'go', 'had', 'regarding', 'too', '‘ve', 'should', 'amount', 'would', 'almost', 'whence', 'together', 'nobody', 'whether', 'whereupon', 'whither', 'his', 'seems', 'you', 'call', 'noone', 'down', 'again', 'before', 'until', '’d', 'now', 'give', 'about', 'move', 'seemed', 'through', 'elsewhere', 'am', 'made', 'once', 'last', "'ll", 'whole', 'those', 'unless', 'its', 'might', 'never', 'formerly', 'an', 'former', 'besides', 'i', 'always', 'them', 'must', 'side', 'within', 'however', 'our', 'put', 'five', 'ourselves', 'toward', 'sixty', 'somewhere', 'behind', 'none', 'nine', 'others', 'third', 'next', 'some', "'m", 'yourself', 'whenever', 'yet', "'d", 'as', 'why', 'few', 'much', 'whoever', 'n’t', 'seem', 'it', 'of', 'front', 'fifty', '’ll', 'someone', 'other', 'do', 'doing', 'herein', 'thence', 'else', 'full', 'further', 'how', 'rather', 'both', 'after', 'who', 'across', 'himself', 'neither

In [31]:
umap_model = UMAP(n_neighbors=5, min_dist=0.1, n_components=2,random_state=42,metric='cosine') #umap_model 정의
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean',cluster_selection_method='eom', prediction_data=True)#hdbscan_model 정의

embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# nr_topics ='auto' auto 제거시 더 많은 토픽을 볼 수 있음 (auto시 자동으로 유사도가 높은 토픽끼리 합침)
topic_model = BERTopic(verbose=True, embedding_model=embedding_model,top_n_words=5,min_topic_size=5,umap_model=umap_model,hdbscan_model=hdbscan_model)#,seed_topic_list=seed_topic_list)
#데이터 좀 많을 떄 min_topic_size=10 , min_cluster_Size=6 등으로 조절

In [32]:
topics, probabillities= topic_model.fit_transform(article_78)

100%|██████████| 100/100 [00:00<00:00, 159.96it/s]
2023-09-29 09:01:21,141 - BERTopic - Transformed documents to Embeddings
2023-09-29 09:01:23,009 - BERTopic - Reduced dimensionality
2023-09-29 09:01:23,047 - BERTopic - Clustered reduced embeddings


In [33]:
#(spacy) tag_ N으로 시작하고 정의한 불용어 사전에 포함되지 않고 entity에서 TIME, CARDINAL, DATE, PERSON은 제거
#그리고 소문자로 변환해서 조건을 만족하는 값만 반환
#output을 뽑아낸 topics 변수에서 사후 텍스트 전처리 진행
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N','V'] and token.lemma_.lower() not in ENGLISH_STOP_WORDS and token.ent_type_ not in ['TIME','CARDINAL','DATE']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())

    return words

In [34]:
#클래스 기반 TF-IDF로 토픽을 추출
vectorizer_model = TfidfVectorizer(tokenizer=tokenizer,stop_words="english", ngram_range=(1, 2),min_df=2)
topic_model.update_topics(article_78, topics, vectorizer_model=vectorizer_model)#위에서 정의한 사용자 정의 tokenzier를 활용해 불용어 제거

In [35]:
topic_model.get_topic_info() #전체 정보

,Topic,Count,Name,Representation,Representative_Docs
0,-1,17,-1_music_ai_user_protection,"[music, ai, user, protection, spotify, salesfo...",[Dr. Scott Gottlieb is a CNBC contributor and ...
1,0,13,0_delivery_grab_food_food delivery,"[delivery, grab, food, food delivery, bank, si...","[Swedish buy now, pay later firm Klarna reduce..."
2,1,8,1_semiconductor_china_venture_export,"[semiconductor, china, venture, export, letter...",[Foxconn logo displayed on a phone screen and ...
3,2,7,2_iphone_smartphone_apple_price,"[iphone, smartphone, apple, price, phone, devi...",[In this article AAPL Follow your favorite sto...
4,3,6,3_zoom_refinitiv_year_loss,"[zoom, refinitiv, year, loss, accord refinitiv...",[Uber CEO Dara Khosrowshahi addresses the audi...
5,4,6,4_tesla_employee_amazon_office,"[tesla, employee, amazon, office, driver, dron...",[An Amazon delivery drone is on display at Ama...
6,5,5,5_settlement_breach_microsoft_corp,"[settlement, breach, microsoft, corp, patient,...","[Sam Hazen, CEO of HCA Healthcare, speaks abou..."
7,6,5,6_microsoft_regulator_judge_competition,"[microsoft, regulator, judge, competition, ftc...","[watch now\n\nMicrosoft says it ""really tried""..."
8,7,5,7_china_ban_rule_bot,"[china, ban, rule, bot, alibaba, administratio...",[An Alibaba Group sign is seen at the World Ar...
9,8,5,8_chip_tsmc_arm_nvidia,"[chip, tsmc, arm, nvidia, revenue, morgan stan...",[Nvidia shares climbed 6% in extended trading ...


In [36]:
topic_model.get_topic(0)# 토픽 하나씩 뽑아서 키워드 체크

[('delivery', 0.04822180760506062),
 ('grab', 0.0462328383895836),
 ('food', 0.044922732549681814),
 ('food delivery', 0.04344771089555203),
 ('bank', 0.029284165224354793),
 ('singapore', 0.029035685242125703),
 ('say', 0.02829959173733761),
 ('company', 0.02608447324840134),
 ('discount', 0.02544445074158479),
 ('wang', 0.025149473346963585)]

In [37]:
similar_topics, similarity = topic_model.find_topics("bitcoin", top_n=5); similar_topics # 특정단어와 가장 연관성 높은 토픽 찾기

[4, 7, 9, -1, 3]

In [42]:
similar_topics, similarity = topic_model.find_topics("battery", top_n=5); similar_topics # 특정단어와 가장 연관성 높은 토픽 찾기

[2, 15, 4, 9, 3]

In [48]:
topic_model.get_representative_docs(0)[2] # 토픽에 속한 문서 내용 최대 3개까지 볼 수 있음

'Instacart, the grocery delivery company that slashed its valuation during last year\'s market slide, filed its paperwork to go public on Friday in what\'s poised to be the first significant venture-backed tech IPO since December 2021.\n\nThe stock will be listed on the Nasdaq under the ticker symbol "CART." In its prospectus, the company said net income totaled $114 million, while revenue in the latest quarter hit $716 million, a 15% increase from the year-ago period. Instacart has now been profitable for five straight quarters, according to the filing. PepsiCo has agreed to purchase $175 million of the company\'s stock in a private placement.\n\nInstacart said it will continue to focus on incorporating artificial intelligence and machine learning features into the platform, and that the company expects to "rely on AIML solutions to help drive future growth in our business." In May, Instacart said it was leaning into the generative AI boom with Ask Instacart, a search tool that aims t

In [49]:
topic_model.visualize_topics()

In [50]:
topic_model.visualize_barchart(top_n_topics=10)

# 전체 데이터 토픽모델링

In [51]:
# 7월과 8월 데이터 필터링
df78_data = df[(df['date'].dt.month == 7) | (df['date'].dt.month == 8)]

# 결과 출력
df78_data

,title,date,category,key_points,text,url
2,"Chinese EV maker Xpeng expects cost cuts, Volk...",2023-08-21,Tech,Chinese EV maker Xpeng expects cost cuts and i...,Xpeng expects cost cuts and its Volkswagen par...,https://www.cnbc.com/2023/08/21/chinas-xpeng-e...
10,Threads has lured some creators who never love...,2023-07-21,Tech Drivers,Creators and influencers are exploring whether...,In this article META Follow your favorite stoc...,https://www.cnbc.com/2023/07/21/instagram-thre...
11,Bill Gates used to think this habit was lazy a...,2023-08-03,,,Bill Gates didn't get much sleep while running...,https://www.cnbc.com/2023/08/03/bill-gates-sle...
13,Parents who raise successful kids use these 4 ...,2023-07-14,,When they started their chores but didn't fini...,Your word choice when talking to your kids is ...,https://www.cnbc.com/2023/07/14/parents-who-ra...
16,HCA Healthcare patient data stolen and for sal...,2023-07-10,Tech,HCA Healthcare patient data has been hacked an...,"Sam Hazen, CEO of HCA Healthcare, speaks about...",https://www.cnbc.com/2023/07/10/hca-healthcare...
...,...,...,...,...,...,...
2075,Wheat prices soar for a third day as Russia-Uk...,2023-07-20,Europe News,Wheat prices rose for a third straight day on ...,"Known as the world's ""bread basket,"" Ukraine g...",https://www.cnbc.com/2023/07/20/wheat-prices-s...
2079,"Delta posts record quarterly earnings, hikes f...",2023-07-13,Airlines,Delta had record revenue and earnings for the ...,"""I think the trends that we've seen this year ...",https://www.cnbc.com/2023/07/13/delta-air-line...
2080,Europe braces for yet another heatwave as Gree...,2023-07-21,Sustainable Future,Europe is set for yet another heatwave in the ...,"Atop the Acropolis ancient hill, tourists visi...",https://www.cnbc.com/2023/07/21/europe-heatwav...
2097,Nasdaq drops more than 2% in worst day since F...,2023-08-01,,,The last time the U.S. got a downgrade from a ...,https://www.cnbc.com/2023/08/01/stock-market-t...


In [52]:
#text 추출
all_article_78 = df78_data['text']
all_article_78

2       Xpeng expects cost cuts and its Volkswagen par...
10      In this article META Follow your favorite stoc...
11      Bill Gates didn't get much sleep while running...
13      Your word choice when talking to your kids is ...
16      Sam Hazen, CEO of HCA Healthcare, speaks about...
                              ...                        
2075    Known as the world's "bread basket," Ukraine g...
2079    "I think the trends that we've seen this year ...
2080    Atop the Acropolis ancient hill, tourists visi...
2097    The last time the U.S. got a downgrade from a ...
2099    (L)A SpaceX Falcon Heavy rocket stands at pad ...
Name: text, Length: 499, dtype: object

In [53]:
#형태소 분석 모델 정의 (spacy)
nlp = spacy.load("en_core_web_md")#trf 모델로 변경가능
spacy_stopwords_list = list(nlp.Defaults.stop_words)
nlp.max_length = 40000000

In [66]:
umap_model = UMAP(n_neighbors=5, min_dist=0.1, n_components=2,random_state=42,metric='cosine') #umap_model 정의
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean',cluster_selection_method='eom', prediction_data=True)#hdbscan_model 정의

embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# nr_topics ='auto' auto 제거시 더 많은 토픽을 볼 수 있음 (auto시 자동으로 유사도가 높은 토픽끼리 합침)
topic_model = BERTopic(verbose=True, embedding_model=embedding_model, nr_topics = 20, top_n_words=5,min_topic_size=5,umap_model=umap_model,hdbscan_model=hdbscan_model)#,seed_topic_list=seed_topic_list)


In [67]:
topics, probabillities= topic_model.fit_transform(all_article_78)

100%|██████████| 499/499 [00:03<00:00, 140.04it/s]
2023-09-29 09:36:14,697 - BERTopic - Transformed documents to Embeddings
2023-09-29 09:36:16,429 - BERTopic - Reduced dimensionality
2023-09-29 09:36:16,467 - BERTopic - Clustered reduced embeddings
2023-09-29 09:36:17,602 - BERTopic - Reduced number of topics from 86 to 20


In [68]:
#(spacy) tag_ N으로 시작하고 정의한 불용어 사전에 포함되지 않고 entity에서 TIME, CARDINAL, DATE, PERSON, 'MONEY','GPE'은 제거
#그리고 소문자로 변환해서 조건을 만족하는 값만 반환
#output을 뽑아낸 topics 변수에서 사후 텍스트 전처리 진행
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N','V'] and token.lemma_.lower() not in ENGLISH_STOP_WORDS and token.ent_type_ not in ['TIME','CARDINAL','DATE']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())

    return words

In [69]:
vectorizer_model = TfidfVectorizer(tokenizer=tokenizer,stop_words="english", ngram_range=(1, 2),min_df=2)
topic_model.update_topics(all_article_78, topics, vectorizer_model=vectorizer_model)#위에서 정의한 사용자 정의 tokenzier를 활용해 불용어 제거

In [70]:
topic_model.get_topic_info() #전체 정보

,Topic,Count,Name,Representation,Representative_Docs
0,-1,61,-1_company_say_bank_business,"[company, say, bank, business, david, datum, r...","[Shoppers at Brickell City Centre in Miami, Fl..."
1,0,163,0_bank_earning_rate_share,"[bank, earning, rate, share, stock, revenue, f...",[watch now\n\nFederal Reserve Chair Jerome Pow...
2,1,66,1_china_climate_energy_oil,"[china, climate, energy, oil, emission, u.s, g...",[Europe was gripped by punishing heat waves in...
3,2,46,2_ai_google_thread_chatgpt,"[ai, google, thread, chatgpt, company, twitter...",[In this article META Follow your favorite sto...
4,3,35,3_music_streaming_sport_league,"[music, streaming, sport, league, disney, acti...","[""This Court's responsibility in this case is ..."
5,4,25,4_happiness_sleep_say_make,"[happiness, sleep, say, make, life, harvard, p...",[When I first started the Harvard course about...
6,5,19,5_trump_trial_iowa_case,"[trump, trial, iowa, case, election, president...","[Former New York City Mayor Rudy Giuliani, an ..."
7,6,17,6_iran_drone_ukraine_minister,"[iran, drone, ukraine, minister, singapore, po...",[SINGAPORE - SEPTEMBER 11: Prime Minister and ...
8,7,11,7_patient_drug_va_doctor,"[patient, drug, va, doctor, device, wegovy, pr...",[In this article MSFT Follow your favorite sto...
9,8,8,8_state_home_housing_score,"[state, home, housing, score, home price, livi...","[When it comes to your finances, where you liv..."


In [72]:
topic_model.get_topic(2)# 토픽 하나씩 뽑아서 키워드 체크

[('ai', 0.06203454592086476),
 ('google', 0.02563737593499035),
 ('thread', 0.025265464916490663),
 ('chatgpt', 0.02503564579434265),
 ('company', 0.02307452650440925),
 ('twitter', 0.022951924419637726),
 ('app', 0.021260671924218735),
 ('user', 0.02122510577871718),
 ('say', 0.02086198513107618),
 ('chatbot', 0.019629542986504992)]

In [73]:
topic_model.get_topic(1)# 토픽 하나씩 뽑아서 키워드 체크

[('china', 0.03544430837595384),
 ('climate', 0.02765968546444564),
 ('energy', 0.026918552763506968),
 ('oil', 0.021968673958455292),
 ('emission', 0.01846682662457301),
 ('u.s', 0.018376817204693478),
 ('gas', 0.017759427529229606),
 ('say', 0.01755388478777851),
 ('carbon', 0.01744775313275802),
 ('water', 0.01683646732399133)]

In [76]:
topic_model.get_topic(3)# 토픽 하나씩 뽑아서 키워드 체크

[('music', 0.03041347550488482),
 ('streaming', 0.027475323200114567),
 ('sport', 0.02690656292961176),
 ('league', 0.0266683716894311),
 ('disney', 0.025892032350177025),
 ('activision', 0.025889291382558917),
 ('netflix', 0.02344767455191793),
 ('espn', 0.02128148994974195),
 ('microsoft', 0.020455265195531803),
 ('iger', 0.019615129318754584)]

In [74]:
topic_model.get_topic(6)# 토픽 하나씩 뽑아서 키워드 체크

[('iran', 0.05309483487999539),
 ('drone', 0.042303948292166244),
 ('ukraine', 0.04152435456464965),
 ('minister', 0.034912955309708144),
 ('singapore', 0.030115268527596696),
 ('police', 0.028314212395173493),
 ('government', 0.025488487058866766),
 ('bank', 0.025191069559088734),
 ('russia', 0.025119788567076577),
 ('tan', 0.02407619677776966)]

In [65]:
topic_model.visualize_topics()

In [71]:
topic_model.visualize_topics()

In [75]:
topic_model.visualize_barchart(top_n_topics=10)